In [ ]:
# ! pip install beautifulsoup4

https://kenpom.com/

In [ ]:
import os
import sys
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
def html2csv(year):
    path = f'{year}.html'
    # empty list
    data = []
    
    # for getting the header from
    # the HTML file
    list_header = []
    soup = BeautifulSoup(open(path),'html.parser')
    header = soup.find_all("table")[0].find("tr")
    
    for items in header:
        try:
            list_header.append(items.get_text())
        except:
            continue
    
    # for getting the data 
    HTML_data = soup.find_all("table")[0].find_all("tr")[1:]
    
    for element in HTML_data:
        sub_data = []
        for sub_element in element:
            try:
                sub_data.append(sub_element.get_text())
            except:
                continue
        data.append(sub_data)
    
    columns = [
        'RK',
        'TEAM',
        'CONF',
        'W-L', 
        'ADJEM', 
        'ADJOE', 'ADJOE_R', 
        'ADJDE', 'ADJDE_R',
        'ADJ_T', 'ADJ_T_R',
        'Luck', 'Luck_R',
        'SOS', 'SOS_R',
        'OppO', 'OppO_R',
        'OppD', 'OppD_R',
        'NCSOS', 'NCSOS_R'
    ]
    int_cols = [
        'RK',
        'ADJOE_R', 
        'ADJDE_R',
        'ADJ_T_R',
        'Luck_R',
        'SOS_R',
        'OppO_R',
        'OppD_R',
        'NCSOS_R'
    ]
    float_cols = [
        'ADJEM', 
        'ADJOE', 
        'ADJDE', 
        'ADJ_T', 
        'Luck', 
        'SOS', 
        'OppO', 
        'OppD', 
        'NCSOS', 
    ]
    
    for i in range(len(data)):
        data[i] = [x for x in data[i] if x != "\n" and x != ""]
        data[i] = [x if '*' not in x else ''.join(x.split(' ')[:-1]) for x in data[i]]
    data = [x for x in data if len(x) == 21]

    df = pd.DataFrame(data = data, columns = columns)
    def fix_ints(s):
        return ' '.join([x for x in ''.join([i for i in s if not i.isdigit()]).split(' ') if x != ''])
    df['TEAM'] = df['TEAM'].apply(fix_ints)

    df[int_cols] = df[int_cols].astype(int)
    df[float_cols] = df[float_cols].astype(float)
    df["YEAR"] = year
    def get_wins(s):
        return int(s.split('-')[0])
    def get_losses(s):
        return int(s.split('-')[1])
    def get_games(s):
        return int(s.split('-')[0]) + int(s.split('-')[1])
    df['W'] = df['W-L'].apply(get_wins)
    df['L'] = df['W-L'].apply(get_losses)
    df['G'] = df['W-L'].apply(get_games)
    return df

In [ ]:
all_df = None
for i in range(23):
    df = html2csv(2002 + i)
    all_df = pd.concat((all_df, df), axis=0)
all_df

In [ ]:
all_df.to_csv('kenpom.csv', index = False)

In [ ]:
df.dtypes

In [ ]:
df